In [ ]:
%load_ext autoreload
%autoreload 2

from dsapol import parsec
from dsapol import polbeamform
from dsapol import polcal
import numpy as np
from matplotlib import pyplot as plt
from dsapol import dsapol
from scipy.signal import correlate
from scipy.signal import savgol_filter as sf
from scipy.signal import convolve
from scipy.signal import fftconvolve
from scipy.ndimage import convolve1d
from scipy.signal import peak_widths
from scipy.stats import chi
from scipy.stats import norm
import copy
import glob
import csv


from numpy.ma import masked_array as ma
from scipy.stats import kstest
from scipy.optimize import curve_fit


from scipy.signal import find_peaks
from scipy.signal import peak_widths
import copy
import numpy as np

from sigpyproc import FilReader
from sigpyproc.Filterbank import FilterbankBlock
from sigpyproc.Header import Header
from matplotlib import pyplot as plt
import pylab
import pickle
import json
from scipy.interpolate import interp1d
from scipy.stats import chi2
from scipy.stats import chi
from scipy.signal import savgol_filter as sf
from scipy.signal import convolve
from scipy.ndimage import convolve1d
from RMtools_1D.do_RMsynth_1D import run_rmsynth
from RMtools_1D.do_RMclean_1D import run_rmclean
from RMtools_1D.do_QUfit_1D_mnest import run_qufit
from astropy.time import Time
from astropy.coordinates import EarthLocation
import astropy.units as u

import pandas as pd
import mercury as mr

In [ ]:

app = mr.App(title="(V2) Polarization Analysis Dashboard", description="With this dashboard, you can conduct RM synthesis, get polarization fractions, and conduct de-dispersion tuning.")

# (0) Load Data

In [ ]:
#if parsec.state_dict["current_state"] == parsec.state_map['load']:

#widgets for data loading dashboard
mr.Markdown("### (0) Load Data")
mr.Note(text="# Load Data Parameters")


frbfiles = parsec.get_frbfiles()
frbfiles_menu = mr.Select(label="FRB (candname_nickname):", 
                            value=frbfiles[0], choices=frbfiles,hidden=False)
base_n_t_slider = mr.Slider( #time downsampling factor
    value=1,
    min=1, 
    max=128, 
    step=1, 
    label=r'base $n_t$'
,hidden=False)
base_logn_f_slider = mr.Slider( #freq downsampling factor
    value=0,
    min=0, 
    max=10, 
    step=1, 
    label=r'base ${\rm log_{2}}({\rm n_f})$'
,hidden=False)
logibox_slider_init = mr.Slider(
    value=0,
    min=0, 
    max=6, 
    step=1, 
    label=r'${\rm log_{2}}({\rm ibox})$'
,hidden=False)

buff_L_slider_init = mr.Slider(
    value=1,
    min=1, 
    max=128, 
    step=1, 
    label=r'${\rm buff}_{L}$'
,hidden=False)
buff_R_slider_init = mr.Slider(
    value=1,
    min=1, 
    max=128, 
    step=1, 
    label=r'${\rm buff}_{R}$'
,hidden=False)



updatebutton = mr.Button(label='Update FRB Params',style='primary',hidden=False)
filbutton = mr.Button(label='Make Filterbanks',style='primary',hidden=False)
loadbutton = mr.Button(label='Load FRB Data',style='primary',hidden=False)
refreshlogbutton = mr.Button(label='Refresh Log',style='primary',hidden=False)

showlog = mr.Checkbox(value=True, label="Show Beamformer Log")



#get parameters for default frb
ids = frbfiles_menu.value[:frbfiles_menu.value.index('_')]
RA = parsec.FRB_RA[parsec.FRB_IDS.index(ids)]
DEC = parsec.FRB_DEC[parsec.FRB_IDS.index(ids)]
ibeam = int(parsec.FRB_BEAM[parsec.FRB_IDS.index(ids)])
mjd = parsec.FRB_mjd[parsec.FRB_IDS.index(ids)]
mjd_display = mr.NumberBox(data=mjd,title=r'MJD')
DMinit = parsec.FRB_DM[parsec.FRB_IDS.index(ids)]


RA_display = mr.NumberBox(data=RA, title=r'RA ($^{\circ}$)')
DEC_display = mr.NumberBox(data=DEC, title=r'DEC ($^{\circ}$)')
ibeam_display = mr.NumberBox(data=ibeam, title=r'ibeam')
mjd_display = mr.NumberBox(data=mjd,title=r'MJD')
DM_input_display = mr.NumberBox(data=DMinit, 
                            title=r'DM (pc/cm$^{{-3}}$)'
                           )



#create load data screen
parsec.load_screen(frbfiles_menu,base_n_t_slider,base_logn_f_slider,logibox_slider_init,
                   buff_L_slider_init,buff_R_slider_init,RA_display,DEC_display,ibeam_display,
                   mjd_display,updatebutton,filbutton,
                   loadbutton)

#display filterbank files if they exist
if 'fils' in parsec.state_dict.keys():
    if len(parsec.state_dict['fils']) == 0:
        mr.Markdown(text=r'##### **Filterbanks:** None Found')
    else:
        mr.Markdown(text=r'##### **Filterbanks:** ')
        for f in parsec.state_dict['fils']:
            mr.Markdown(r'###### -' + str(f))

#display beamformer weights date if exists
if 'bfweights' in parsec.state_dict.keys():
    if parsec.state_dict['bfweights'] is None:
        bfweights_text = r'##### **Beamformer Weights:** None Found'
    else:
        bfweights_text = r'##### **Beamformer Weights:** ' + parsec.state_dict['bfweights']
    bfweights_display = mr.Markdown(text=bfweights_text)


In [ ]:

#if not (parsec.state_dict["current_state"] >= parsec.state_map['load']): raise parsec.StopExecution
mr.Markdown("##### FRB Parameters")
mr.NumberBox([RA_display,DEC_display,ibeam_display])

In [ ]:

#if not (parsec.state_dict["current_state"] >= parsec.state_map['load']): raise parsec.StopExecution
mr.NumberBox([mjd_display,DM_input_display])

In [ ]:
# display the output of the logfile
#if parsec.state_dict["current_state"] == parsec.state_map['load']:
mr.Markdown("##### Log:")
if showlog.value:# or refreshlogbutton.clicked:
    flog = open(polbeamform.logfile,"r")
    logtext = flog.read()
    flog.close()
    mr.Markdown(text=(logtext.replace('\n','\n\n')))


# (1) Dedispersion Tuning

In [ ]:
#if parsec.state_dict["current_state"] == parsec.state_map['dedisp']:

#widgets for DM tuning dashboard
mr.Markdown("### (1) Dedispersion Tuning")
mr.Note(text="# Binning and DM Tuning Parameters")
n_t_slider = mr.Slider( #time downsampling factor
    value=1,
    min=1, 
    max=128, 
    step=1, 
    label='$n_t$'
,hidden=False)
logn_f_slider = mr.Slider( #freq downsampling factor
    value=5,
    min=0, 
    max=10, 
    step=1, 
    label='${\\rm log_{2}}(n_f)$'
,hidden=False)
logwindow_slider = mr.Slider(
    value=5,
    min=5, 
    max=9, 
    step=1, 
    label=r'base ${\rm log_{2}}({\rm window})$'
,hidden=False)
dmstep = parsec.get_min_DM_step(n_t_slider.value)#np.around((32.7e-3)*n_t_slider.value/(4.15)/((1/1.307**2) - (1/1.493**2)),2) #pc/cc
ddm_num = mr.Numeric(value=0, #DM offset
                     step=dmstep,
                     min=-100,max=100,
                     label='$\Delta {\\rm DM}$'
                    ,hidden=False)

DMdonebutton = mr.Button(label='Confirm DM',
                         style='primary',
                        hidden=False)

#if parsec.state_dict["current_state"] >= parsec.state_map['dedisp']:
#create DM screen
parsec.dedisp_screen(n_t_slider,logn_f_slider,logwindow_slider,ddm_num,DM_input_display,DM_new_display,DMdonebutton)



if not (parsec.state_dict["current_state"] >= parsec.state_map['dedisp']): raise parsec.StopExecution
mr.NumberBox([DM_input_display,DM_new_display])
    

In [ ]:
mr.Markdown("##### DM Parameters")
DM_input_display = mr.NumberBox(data=parsec.FRB_DM[parsec.FRB_IDS.index(ids)], 
                                title=r'Input DM (pc/cm$^{{-3}}$)'
                               )
DM_new_display = mr.NumberBox(data=parsec.FRB_DM[parsec.FRB_IDS.index(ids)] + ddm_num.value, 
                              title=r'New DM (pc/cm$^{{-3}}$)'
                             )



# (2) Calibration

In [ ]:
#if parsec.state_dict["current_state"] == parsec.state_map['polcal']:
#widgets for pol cal dashboard
mr.Markdown(" (2) Calibration")
mr.Note(text="# Polarization Calibration Parameters")


#if we want to create new cal solutions from voltage files on T3, choose from here
polcaldates = []
for k in parsec.polcal_dict.keys():
    if 'polcal' not in str(k):
        polcaldates.append(str(k))
polcaldate_create_menu = mr.Select(label="Observation Date for New Polarization Calibration Solution ('YYYY-MM-DD'):", 
                            value="", 
                            choices=[""] + list(polcaldates)
                           ,hidden=False)

polcopybutton = mr.Button(label='Copy T3 Voltages and Beamformer Weights',
                         style='primary',hidden=False)


#if we want to create new cal solutions from voltage files that are ALREADY COPIED TO H23 choose from here
polcalfiles_bf = polcal.get_avail_caldates()
polcaldate_bf_menu = mr.Select(label="Available Polarization Calibration Dates ('YYYY-MM-DD'):", 
                            value="", 
                            choices=[""] + list(polcalfiles_bf)
                           ,hidden=False)


bfcal_button = mr.Button(label='Run Cal Beamformer',style='primary',hidden=False)

#if we want to start from formed beams in scratch directory
polcalfiles_findbeams = polcal.get_beamfinding_files()
polcaldate_findbeams_menu = mr.Select(label="Available Dates With Voltages or Filterbanks on h23 ('YYYY-MM-DD'):", 
                            value=polcalfiles_findbeams[0], 
                            choices=list(polcalfiles_findbeams)
                           ,hidden=False)

findbeams_button = mr.Button(label='Find Beam Numbers',style='primary',hidden=False)
filcalbutton = mr.Button(label='Make Calibrator Filterbanks',style='primary',hidden=False)

refreshlogcalbutton = mr.Button(label='Refresh Log',style='primary',hidden=False)


#if we want to make solution from filterbanks
obs_files,obs_ids = polcal.get_calfil_files('3C48',polcaldate_findbeams_menu.value,'3C48*0')
initval = ''
"""if 'cal_name_3C48_center' in parsec.polcal_dict.keys():
    initval = parsec.polcal_dict['cal_name_3C48_center']
"""
obsid3C48_menu = mr.Select(label="3C48 Cal Observation ID:", 
                            value=initval, 
                            choices=[""] + list(obs_ids)
                           ,hidden=False)
obs_files,obs_ids = polcal.get_calfil_files('3C286',polcaldate_findbeams_menu.value,'3C286*0')
initval = ''
"""if 'cal_name_3C286_center' in parsec.polcal_dict.keys():
    initval = parsec.polcal_dict['cal_name_3C286_center']
"""
obsid3C286_menu = mr.Select(label="3C286 Cal Observation ID:", 
                            value=initval, 
                            choices=[""] + list(obs_ids)
                           ,hidden=False)

mr.Note(text=r'## $|g_{yy}|$')


peakheight_slider = mr.Slider( 
    value=2,
    min=1, 
    max=10, 
    step=1, 
    label=r'Peak Height'
,hidden=False)
peakwidth_slider = mr.Slider( 
    value=10,
    min=1, 
    max=20, 
    step=1, 
    label=r'Peak Width'
,hidden=False)
sfflag =  mr.Checkbox(value=False, label="Savgol Filter",hidden=False)
sf_window_weight_cals = mr.Slider( 
    value=255,
    min=3, 
    max=510, 
    step=2, 
    label=r'$sf_{ww}$'
,hidden=False)
sf_order_cals = mr.Slider( 
    value=5,
    min=1, 
    max=10, 
    step=1, 
    label=r'$sf_{o}$'
,hidden=False)
polyfitflag =  mr.Checkbox(value=False, label="Fit with Polynomial",hidden=False)
polyfitorder_slider = mr.Slider( 
    value=5,
    min=1, 
    max=9, 
    step=1, 
    label=r'Polynomial Fit Degree'
,hidden=False)
edgefreq_slider = mr.Slider( 
    value=1370,
    min=1311.25000003072, 
    max=1498.75, 
    step=30.4e-3, 
    label=r'Edge Frequency (MHz) Cutoff (for Fitting Piecewise Solution)'
,hidden=False)
breakfreq_slider = mr.Slider( 
    value=1370,
    min=1311.25000003072, 
    max=1498.75, 
    step=30.4e-3, 
    label=r'Break Frequency (MHz) Cutoff (for Stitching Piecewise Solution)'
,hidden=False)

mr.Note(text=r'## $|g_{xx}|/|g_{yy}|$')
ratio_peakheight_slider = mr.Slider( 
    value=3,
    min=1, 
    max=10, 
    step=1, 
    label=r'Peak Height'
,hidden=False)
ratio_peakwidth_slider = mr.Slider( 
    value=10,
    min=1, 
    max=20, 
    step=1, 
    label=r'Peak Width'
,hidden=False)
ratio_sfflag =  mr.Checkbox(value=False, label="Savgol Filter",hidden=False)
ratio_sf_window_weight_cals = mr.Slider( 
    value=257,
    min=3, 
    max=510, 
    step=2, 
    label=r'$sf_{ww}$'
,hidden=False)
ratio_sf_order_cals = mr.Slider( 
    value=5,
    min=1, 
    max=10, 
    step=1, 
    label=r'$sf_{o}$'
,hidden=False)
ratio_polyfitflag =  mr.Checkbox(value=False, label="Fit with Polynomial",hidden=False)
ratio_polyfitorder_slider = mr.Slider( 
    value=5,
    min=1, 
    max=9, 
    step=1, 
    label=r'Polynomial Fit Degree'
,hidden=False)
ratio_edgefreq_slider = mr.Slider( 
    value=1360,
    min=1311.25000003072, 
    max=1498.75, 
    step=30.4e-3, 
    label=r'Edge Frequency (MHz) Cutoff (for Fitting Piecewise Solution)'
,hidden=False)
ratio_breakfreq_slider = mr.Slider( 
    value=1360,
    min=1311.25000003072, 
    max=1498.75, 
    step=30.4e-3, 
    label=r'Break Frequency (MHz) Cutoff (for Stitching Piecewise Solution)'
,hidden=False)

mr.Note(text=r'## $\angle g_{xx} - \angle g_{yy}$')
phase_peakheight_slider = mr.Slider( 
    value=3,
    min=1, 
    max=10, 
    step=1, 
    label=r'Peak Height'
,hidden=False)
phase_peakwidth_slider = mr.Slider( 
    value=10,
    min=1, 
    max=20, 
    step=1, 
    label=r'Peak Width'
,hidden=True)
phase_sfflag =  mr.Checkbox(value=False, label="Savgol Filter",hidden=False)
phase_sf_window_weight_cals = mr.Slider( 
    value=255,
    min=3, 
    max=510, 
    step=2, 
    label=r'$sf_{ww}$'
,hidden=False)
phase_sf_order_cals = mr.Slider( 
    value=5,
    min=1, 
    max=10, 
    step=1, 
    label=r'$sf_{o}$'
,hidden=False)
phase_polyfitflag =  mr.Checkbox(value=False, label="Fit with Polynomial",hidden=False)
phase_polyfitorder_slider = mr.Slider( 
    value=5,
    min=1, 
    max=9, 
    step=1, 
    label=r'Polynomial Fit Degree'
,hidden=False)

savecalsolnbutton = mr.Button(label='Save Cal Solution',
                         style='primary',hidden=False)

#if pol cal solution already exists
polcalfiles = glob.glob(parsec.default_path + 'POLCAL_PARAMETERS_*csv')
polcalfiles = [polcalfiles[i][polcalfiles[i].index('POLCAL'):] for i in range(len(polcalfiles))]
polcaldate_menu = mr.Select(label="Polarization Calibration File ('YY-MM-DD'):", 
                            value="", 
                            choices=[""] + polcalfiles
                           ,hidden=False)

polcalbutton = mr.Button(label='Calibrate',
                         style='primary',hidden=False)


showlogcal = mr.Checkbox(value=False, label="Show Pol Cal Log",hidden=False)

ParA_display = mr.NumberBox(data=np.nan, title=r'Parallactic Angle ($^{\circ}$)')

    
    

#create pol cal screen
beam_dict_3C48,beam_dict_3C286 = parsec.polcal_screen(polcaldate_menu,polcaldate_create_menu,polcaldate_bf_menu,polcaldate_findbeams_menu,obsid3C48_menu,obsid3C286_menu,
                                                     polcalbutton,polcopybutton,bfcal_button,findbeams_button,filcalbutton,ParA_display,
                                                     edgefreq_slider,breakfreq_slider,sf_window_weight_cals,sf_order_cals,peakheight_slider,peakwidth_slider,polyfitorder_slider,
                                                     ratio_edgefreq_slider,ratio_breakfreq_slider,ratio_sf_window_weight_cals,ratio_sf_order_cals,ratio_peakheight_slider,ratio_peakwidth_slider,ratio_polyfitorder_slider,
                                                     phase_sf_window_weight_cals,phase_sf_order_cals,phase_peakheight_slider,phase_peakwidth_slider,phase_polyfitorder_slider,savecalsolnbutton,
                                                     sfflag,polyfitflag,ratio_sfflag,ratio_polyfitflag,phase_sfflag,phase_polyfitflag)
    

In [ ]:
#table of new pol cal data
if polcaldate_create_menu.value != "":
    mr.Table(data=parsec.df_polcal, width="200px")


In [ ]:

#display voltages and beamformer weight files for current selection
if polcaldate_bf_menu.value != "":
    #display filterbank files if they exist
    if 'polcal_avail_3C48' in parsec.polcal_dict.keys():
        if len(parsec.polcal_dict['polcal_avail_3C48']) == 0:
            mr.Markdown(text=r'###### **3C48 Voltages:** None Found')
        else:
            mr.Markdown(text=r'###### **3C48 Voltages:** ')
            for f in parsec.state_dict['polcal_avail_3C48']:
                mr.Markdown(r'- ######' + f)
    if 'polcal_avail_3C286' in parsec.polcal_dict.keys():
        if len(parsec.polcal_dict['polcal_avail_3C286']) == 0:
            mr.Markdown(text=r'###### **3C286 Voltages:** None Found')
        else:
            mr.Markdown(text=r'###### **3C286 Voltages:** ')
            for f in parsec.polcal_dict['polcal_avail_3C286']:
                mr.Markdown(r'- ######' + f)


In [ ]:

#display beamformer weights date if exists
if 'polcal_avail_bf_3C48' in parsec.polcal_dict.keys():
    if len(parsec.polcal_dict['polcal_avail_bf_3C48']) == 0:
        mr.Markdown(text=r'##### **3C48 Beamformer Weights:** None Found')
    else:
        mr.Markdown(text=r'##### **3C48 Beamformer Weights:** ')
        for f in parsec.state_dict['polcal_avail_bf_3C48']:
            mr.Markdown(r'- #####' + f)
if 'polcal_avail_bf_3C286' in parsec.polcal_dict.keys():
    if len(parsec.polcal_dict['polcal_avail_bf_3C286']) == 0:
        mr.Markdown(text=r'##### **3C286 Beamformer Weights:** None Found')
    else:
        mr.Markdown(text=r'##### **3C286 Beamformer Weights:** ')
        for f in parsec.polcal_dict['polcal_avail_bf_3C286']:
            mr.Markdown(r'- #####' + f)


In [ ]:

#display beam numbers for each cal pass
mr.Table(data=parsec.df_beams,width="200px") 


In [ ]:

#display the filterbank and beamformer weight files for selected observation ID

#3C48 
if obsid3C48_menu.value != '':
    obs_files,obs_ids = polcal.get_calfil_files('3C48',polcaldate_findbeams_menu.value,obsid3C48_menu.value)
    mr.Markdown(text=r'##### **' + obsid3C48_menu.value + ' Filterbanks:**')
    for f in obs_files:
        mr.Markdown(r'- #####' + f)
else:
    mr.Markdown(text = r'##### **' + obsid3C48_menu.value + ' Filterbanks:** None Found')


In [ ]:

if obsid3C48_menu.value in parsec.df_beams.index.tolist():#beam_dict_3C48.keys():
    #print(beam_dict_3C48.keys())
    mr.Markdown(text=r'##### **' + obsid3C48_menu.value + ' Beamformer Weights:**')
    #print(polcal.bfweights_output_path + "*" + polcal.VLANAME_DICT['3C48'] + "*" + parsec.df_beams['beamformer weights'].loc[obsid3C48_menu.value][:19] + "*")
    bf_files = glob.glob(polcal.bfweights_output_path + "*" + polcal.VLANAME_DICT['3C48'] + "*" + parsec.df_beams['beamformer weights'].loc[obsid3C48_menu.value][:19] + "*")
    for f in bf_files:
        mr.Markdown(text = r'- #####' + f)
else:
    mr.Markdown(text = r'##### **' + obsid3C48_menu.value + ' Beamformer Weights:** None Found')


In [ ]:

#3C286
if obsid3C286_menu.value != '':
    obs_files,obs_ids = polcal.get_calfil_files('3C286',polcaldate_findbeams_menu.value,obsid3C286_menu.value)
    mr.Markdown(text=r'##### **' + obsid3C286_menu.value + ' Filterbanks:**')
    for f in obs_files:
        mr.Markdown(text = r'- #####' + f)
else:
    mr.Markdown(text = r'##### **' + obsid3C286_menu.value + ' Filterbanks:** None Found')


In [ ]:

if obsid3C286_menu.value in parsec.df_beams.index.tolist():
    mr.Markdown(text=r'##### **' + obsid3C286_menu.value + ' Beamformer Weights:**')
    bf_files = glob.glob(polcal.bfweights_output_path + "*" + polcal.VLANAME_DICT['3C286'] + "*" + parsec.df_beams['beamformer weights'].loc[obsid3C286_menu.value][:19] + "*")
    #print(polcal.bfweights_output_path + "*" + polcal.VLANAME_DICT['3C286'] + "*" + parsec.df_beams['beamformer weights'].loc[obsid3C286_menu.value][:19] +  "*")
    for f in bf_files:
        mr.Markdown(text = r'- #####' + f)
else:
    mr.Markdown(text=r'##### **' + obsid3C286_menu.value + ' Beamformer Weights:** None Found')



In [ ]:
if not (parsec.state_dict["current_state"] >= parsec.state_map['polcal']): raise parsec.StopExecution
ParA_display

In [ ]:
# display the output of the logfile
if showlogcal.value:# or refreshlogbutton.clicked:
    flog = open(polcal.logfile,"r")
    logtext = flog.read()
    flog.close()
    mr.Markdown(text=(logtext.replace('\n','\n\n')))


# (3) Filter Weights

In [ ]:
if parsec.state_dict["current_state"] == parsec.state_map['filter']:
    #widgets for filter weights dashboard
    mr.Markdown(" (3) Filter Weights")
    mr.Note(text="# Filter Weight Options")

    ncomps_num = mr.Numeric(value=1, 
                         step=1,
                         min=1,max=5,
                         label='Number of Components'
                        ,hidden=False)
    comprange_slider = mr.Range(value=[0,1],  #ms
                            min=0, 
                            max=50, 
                            label="Lower and Upper Limit of Component (ms from window start)", 
                            step=32.7e-3
                    ,hidden=False)
    
    avger_w_slider = mr.Slider(
        value=1,
        min=1, 
        max=32, 
        step=1, 
        label=r'$n_{tw}$'
    ,hidden=False)
    sf_window_weights_slider = mr.Slider(
        value=3,
        min=3, 
        max=33, 
        step=2, 
        label=r'$sf_{ww}$'
    ,hidden=False)
    
    nextcompbutton = mr.Button(label='Next Component',
                             style='primary',hidden=False)

    donecompbutton = mr.Button(label='Done',
                             style='primary',hidden=False)



    #create pol cal screen
    parsec.filter_screen(n_t_slider,logn_f_slider,logwindow_slider,logibox_slider,
                         buff_L_slider,buff_R_slider,ncomps_num,comprange_slider,nextcompbutton,donecompbutton,
                        avger_w_slider,sf_window_weights_slider)
    mr.Table(data=parsec.df, width="200px")#, text_align="center")
    
if not (parsec.state_dict["current_state"] >= parsec.state_map['filter']): raise parsec.StopExecution
mr.NumberBox(data=parsec.state_dict["current_state"], 
                              title=r'Current Screen Index'
                             )